In [1]:
import sklearn
from sklearn import svm, preprocessing, cross_validation
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, linear_kernel 
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.naive_bayes import GaussianNB
from sklearn.learning_curve import learning_curve
import random
import numpy as np
import gensim
import csv
import numpy as np
import pandas as pd
import timeit
import nltk
import itertools
import glove
from collections import defaultdict
import pandas as pd
from nltk.stem import *
from nltk import word_tokenize, ngrams
import re
from gensim import corpora
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

#### Loading Stopwords

In [2]:
nltk.download('stopwords')
stpwds = set(nltk.corpus.stopwords.words("english"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pascalsitbon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Loading Training Set

In [3]:
Training_Set = pd.read_csv('train.csv').dropna()

In [4]:
print(Training_Set.shape)

(404288, 6)


In [5]:
print(Training_Set.iloc[10,:]['question1'])

Method to find separation of slits using fresnel biprism?


In [6]:
clean_data = Training_Set[['question1','question2','is_duplicate']].values

In [7]:
sentences_train = clean_data[:,:2]
labels = clean_data[:,2]

In [8]:
print('sample of data','\n',sentences_train[:10],labels[:10])

sample of data 
 [['What is the step by step guide to invest in share market in india?'
  'What is the step by step guide to invest in share market?']
 ['What is the story of Kohinoor (Koh-i-Noor) Diamond?'
  'What would happen if the Indian government stole the Kohinoor (Koh-i-Noor) diamond back?']
 [ 'How can I increase the speed of my internet connection while using a VPN?'
  'How can Internet speed be increased by hacking through DNS?']
 ['Why am I mentally very lonely? How can I solve it?'
  'Find the remainder when [math]23^{24}[/math] is divided by 24,23?']
 [ 'Which one dissolve in water quikly sugar, salt, methane and carbon di oxide?'
  'Which fish would survive in salt water?']
 [ 'Astrology: I am a Capricorn Sun Cap moon and cap rising...what does that say about me?'
  "I'm a triple Capricorn (Sun, Moon and ascendant in Capricorn) What does this say about me?"]
 ['Should I buy tiago?'
  'What keeps childern active and far from phone and video games?']
 ['How can I be a good

## Doc2Vec Featuring

In [9]:
def doc2vecs_features(sentences_train,nb_epochs=100,alpha=0.025,min_alpha=0.025):
    sentences = []
    stemmer = PorterStemmer()
    for i in range(sentences_train.shape[0]): 
        source_sentence = sentences_train[i,0].lower().split(" ") 
        source_sentence = [token for token in source_sentence if token not in stpwds]
        unigrams_que1 = [stemmer.stem(token) for token in source_sentence]
        sentences.append(unigrams_que1)

        target_sentence = sentences_train[i,1].lower().split(" ") 
        target_sentence= [token for token in target_sentence if token not in stpwds]
        unigrams_que2 = [stemmer.stem(token) for token in target_sentence]
        sentences.append(unigrams_que2)

    texts=sentences.copy()
    documents = []
    ct = 0
    for doc in texts:
        doc = gensim.models.doc2vec.LabeledSentence(words = doc, tags = ['SENT_'+str(ct)])
        ct+=1
        documents.append(doc)
    model = gensim.models.Doc2Vec(alpha=.025, min_alpha=.025, min_count=1,workers=4)
    model.build_vocab(documents)

    for epoch in range(100):
        model.train(documents,total_examples=model.corpus_count,epochs=model.iter)
        
    most_similar_is_duo_1_2 = []
    most_similar_is_duo_2_1 = []
    
    most_similar_score_if_duo_1_2 = []
    most_similar_score_if_duo_2_1 = []
    
    n_similarities = []
    
    for i in range(sentences_train.shape[0]):
        
        most_sim_1 = model.docvecs.most_similar(["SENT_"+str(2*i)])[0]
        most_sim_2 = model.docvecs.most_similar(["SENT_"+str(2*i+1)])[0]
        
        most_similar_is_duo_1_2.append(int(most_sim_1[0] == "SENT_"+str(2*i+1)))
        most_similar_is_duo_2_1.append(int(most_sim_2[0] == "SENT_"+str(2*i)))
        
        most_similar_score_if_duo_1_2.append(int(most_sim_1[0] == "SENT_"+str(2*i+1))*most_sim_1[1])
        most_similar_score_if_duo_2_1.append(int(most_sim_2[0] == "SENT_"+str(2*i))*most_sim_2[1])
        
        n_similarities.append(model.n_similarity(sentences[2*i], sentences[2*i+1]))
                                             
        doc_2_vec_features = np.array([n_similarities,
                                       most_similar_score_if_duo_1_2,
                                       most_similar_score_if_duo_2_1,
                                       most_similar_is_duo_1_2,
                                       most_similar_is_duo_2_1])
                                            
    return doc_2_vec_features
    

In [11]:
print(doc2vecs_features(sentences_train[:10,:]))
# sentences = []
# stemmer = PorterStemmer()
# for i in range(10): 
#     source_sentence = sentences_train[i,0].lower().split(" ") 
#     source_sentence = [token for token in source_sentence if token not in stpwds]
#     unigrams_que1 = [stemmer.stem(token) for token in source_sentence]
#     sentences.append(unigrams_que1)
        
#     target_sentence = sentences_train[i,1].lower().split(" ") 
#     target_sentence= [token for token in target_sentence if token not in stpwds]
#     unigrams_que2 = [stemmer.stem(token) for token in target_sentence]
#     sentences.append(unigrams_que2)

# texts=sentences.copy()
# documents = []
# ct = 0
# for doc in texts:
#     doc = gensim.models.doc2vec.LabeledSentence(words = doc, tags = ['SENT_'+str(ct)])
#     ct+=1
#     documents.append(doc)
# model = gensim.models.Doc2Vec(alpha=.025, min_alpha=.025, min_count=1,workers=4)
# model.build_vocab(documents)

# for epoch in range(100):
#     model.train(documents,total_examples=model.corpus_count,epochs=model.iter)
#     #model.alpha -= 0.002  # decrease the learning rate`
#     #model.min_alpha = model.alpha  # fix the learning rate, no decay

# #model.save("my_model.doc2vec")
# #model_loaded = gensim.models.Doc2Vec.load('my_model.doc2vec')

# print (model.docvecs.most_similar(["SENT_5"]))

2017-05-14 17:15:01,232 : INFO : collecting all words and their counts
2017-05-14 17:15:01,233 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-05-14 17:15:01,235 : INFO : collected 87 word types and 20 unique tags from a corpus of 20 examples and 111 words
2017-05-14 17:15:01,236 : INFO : Loading a fresh vocabulary
2017-05-14 17:15:01,238 : INFO : min_count=1 retains 87 unique words (100% of original 87, drops 0)
2017-05-14 17:15:01,239 : INFO : min_count=1 leaves 111 word corpus (100% of original 111, drops 0)
2017-05-14 17:15:01,241 : INFO : deleting the raw counts dictionary of 87 items
2017-05-14 17:15:01,242 : INFO : sample=0.001 downsamples 87 most-common words
2017-05-14 17:15:01,243 : INFO : downsampling leaves estimated 41 word corpus (37.7% of prior 111)
2017-05-14 17:15:01,245 : INFO : estimated required memory for 87 words and 100 dimensions: 125100 bytes
2017-05-14 17:15:01,246 : INFO : resetting layer weights
2017-05-14 17:15:01,251 : 

[[ 0.9999162   0.98256967  0.96522222  0.63741083  0.41868507  0.97363492
   0.71758412  0.99872238  0.99668273  0.97257676]
 [ 0.99566793  0.86712646  0.93974316  0.          0.          0.85569304
   0.          0.93151903  0.93891066  0.84126103]
 [ 0.99566793  0.86712646  0.9397431   0.          0.          0.85569304
   0.52657533  0.93151903  0.9389106   0.84126103]
 [ 1.          1.          1.          0.          0.          1.          0.
   1.          1.          1.        ]
 [ 1.          1.          1.          0.          0.          1.          1.
   1.          1.          1.        ]]


## Frequency and Hash Tag ID

In [23]:
#frquency questions and hash.
train_orig =  pd.read_csv('train.csv', header=0)
test_orig =  pd.read_csv('test.csv', header=0)

tic0=timeit.default_timer()
df1 = train_orig[['question1']].copy()
df2 = train_orig[['question2']].copy()
df1_test = test_orig[['question1']].copy()
df2_test = test_orig[['question2']].copy()

df2.rename(columns = {'question2':'question1'},inplace=True)
df2_test.rename(columns = {'question2':'question1'},inplace=True)

train_questions = df1.append(df2)
train_questions = train_questions.append(df1_test)
train_questions = train_questions.append(df2_test)
#train_questions.drop_duplicates(subset = ['qid1'],inplace=True)
train_questions.drop_duplicates(subset = ['question1'],inplace=True)

train_questions.reset_index(inplace=True,drop=True)
questions_dict = pd.Series(train_questions.index.values,index=train_questions.question1.values).to_dict()
train_cp = train_orig.copy()
test_cp = test_orig.copy()
train_cp.drop(['qid1','qid2'],axis=1,inplace=True)

test_cp['is_duplicate'] = -1
test_cp.rename(columns={'test_id':'id'},inplace=True)
comb = pd.concat([train_cp,test_cp])

comb['q1_hash'] = comb['question1'].map(questions_dict)
comb['q2_hash'] = comb['question2'].map(questions_dict)

q1_vc = comb.q1_hash.value_counts().to_dict()
q2_vc = comb.q2_hash.value_counts().to_dict()

def try_apply_dict(x,dict_to_apply):
    try:
        return dict_to_apply[x]
    except KeyError:
        return 0
#map to frequency space
comb['q1_freq'] = comb['q1_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))
comb['q2_freq'] = comb['q2_hash'].map(lambda x: try_apply_dict(x,q1_vc) + try_apply_dict(x,q2_vc))

train_comb = comb[comb['is_duplicate'] >= 0][['id','q1_hash','q2_hash','q1_freq','q2_freq','is_duplicate']]
test_comb = comb[comb['is_duplicate'] < 0][['id','q1_hash','q2_hash','q1_freq','q2_freq']]

In [33]:
def select_features(data_set,nb_ex):
    # Select basic features
    tfidf_cosin_sim = []     
    stemmer = PorterStemmer()
    dif_len = []
    common_unigrams_lens = []
    common_unigrams_ratios = []
    common_bigrams_lens = []
    common_bigrams_ratios = []
    common_trigrams_lens = []
    common_trigrams_ratios = []
    similarities = []
    counter = 0
    sentences = []
    for i in range(nb_ex):
        if i%10000== 0:
            print(i)
        # select informations about articles 1 and 2
        
        
        
        source_sentence = data_set[i,0].lower().split(" ") 
        source_sentence = [token for token in source_sentence if token not in stpwds]
        unigrams_que1 = [stemmer.stem(token) for token in source_sentence]
        sentences.append(unigrams_que1)
        
        target_sentence = data_set[i,1].lower().split(" ") 
        target_sentence= [token for token in target_sentence if token not in stpwds]
        unigrams_que2 = [stemmer.stem(token) for token in target_sentence]
        sentences.append(unigrams_que2)

        
        #get unigram features #
        common_unigrams_len = len(set(unigrams_que1).intersection(set(unigrams_que2)))
        common_unigrams_lens.append(common_unigrams_len)
        common_unigrams_ratios.append(float(common_unigrams_len) / max(len(set(unigrams_que1).union(set(unigrams_que2))),1))
        
        # get bigram features #
        bigrams_que1 = [i for i in ngrams(unigrams_que1, 2)]
        bigrams_que2 = [i for i in ngrams(unigrams_que2, 2)]
        common_bigrams_len = len(set(bigrams_que1).intersection(set(bigrams_que2)))
        common_bigrams_lens.append(common_bigrams_len)
        common_bigrams_ratios.append(float(common_bigrams_len) / max(len(set(bigrams_que1).union(set(bigrams_que2))),1))


        # get trigram features #
        trigrams_que1 = [i for i in ngrams(unigrams_que1, 3)]
        trigrams_que2 = [i for i in ngrams(unigrams_que2, 3)]
        common_trigrams_len = len(set(trigrams_que1).intersection(set(trigrams_que2)))
        common_trigrams_lens.append(common_trigrams_len)
        common_trigrams_ratios.append(float(common_trigrams_len) / max(len(set(trigrams_que1).union(set(trigrams_que2))),1))

        dif_len.append(abs(len(source_sentence) - len(target_sentence)))
        
    
    
#     #Cosine Similarity on TF IDF vectors
#     texts = sentences

#     tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, #max_features=n_features,
#                                    stop_words='english')
#     tfidf_sentences= tfidf_vectorizer.fit_transform(data_set[:nb_ex,:].flatten())
    
#     for i in range(nb_ex):
#         tfidf_cosin_sim.append(cosine_similarity(tfidf_sentences[2*i,:], tfidf_sentences[2*i+1,:])[0,0])
  
    
    
# #     #LSI similarities

#     dictionary = corpora.Dictionary(texts)
#     corpus = [dictionary.doc2bow(text) for text in texts]
#     lsi = gensim.models.LsiModel(corpus, id2word=dictionary, num_topics=100)
#     index = gensim.similarities.MatrixSimilarity(lsi[corpus])
#     for i in range(nb_ex):
#         similarities.append(index[lsi[corpus[2*i]]][2*i+1])
    

    # examples as rows, features as columns
    features = np.array([#tfidf_cosin_sim,
                         #similarities,
                         common_unigrams_lens,
                         common_unigrams_ratios,
                         common_bigrams_lens,
                         common_bigrams_ratios,
                         common_trigrams_lens,
                         common_trigrams_ratios,
                         dif_len]).T
    
    
    #normalized_features = preprocessing.scale(features) 
    
    
    
    
    return features

In [34]:
nb_ex = 35000

In [35]:
n_grams_features,y= select_features(sentences_train,nb_ex),labels[:nb_ex].astype(float)#,lsi,index

0
10000
20000
30000


In [36]:
print(n_grams_features.shape)

(35000, 7)


In [37]:
new_features = train_comb[['q1_hash','q2_hash','q1_freq','q2_freq']].iloc[:nb_ex].values

In [39]:
print(new_features.shape)

(35000, 4)


In [40]:
X = np.column_stack([n_grams_features,new_features])
X = preprocessing.scale(X)

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                 test_size=0.3,
                                                 random_state=0)


In [42]:
#clf = LogisticRegression()
clf = RandomForestClassifier(n_estimators=50,max_depth = 5,max_features=4)
clf.fit(X_train,y_train)
print('Training Score:',sklearn.metrics.log_loss(y_train,clf.predict_proba(X_train)[:,1]))
print('Testing Score:',sklearn.metrics.log_loss(y_test,clf.predict_proba(X_test)[:,1]))

Training Score: 0.393326124948
Testing Score: 0.395367589338


In [43]:
clf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features=4, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [44]:
Testing_Set = pd.read_csv('test.csv')
print(Testing_Set.shape)
clean_data_test = Testing_Set[['question1','question2']].values


(2345796, 3)


In [ ]:
def select_basic_features_test(data_set):
    # Select basic features
    tfidf_cosin_sim = []     
    stemmer = PorterStemmer()
    dif_len = []
    common_unigrams_lens = []
    common_unigrams_ratios = []
    common_bigrams_lens = []
    common_bigrams_ratios = []
    common_trigrams_lens = []
    common_trigrams_ratios = []
    similarities = []
    counter = 0
    sentences = []
    for i in range(data_set.shape[0]):
        if i%10000== 0:
            print(i)
        # select informations about articles 1 and 2
        
        
        
        source_sentence = str(data_set[i,0]).lower().split(" ") 
        source_sentence = [token for token in source_sentence if token not in stpwds]
        unigrams_que1 = [stemmer.stem(token) for token in source_sentence]
        sentences.append(unigrams_que1)
        
        target_sentence = str(data_set[i,1]).lower().split(" ") 
        target_sentence= [token for token in target_sentence if token not in stpwds]
        unigrams_que2 = [stemmer.stem(token) for token in target_sentence]
        sentences.append(unigrams_que2)

        
        #get unigram features #
        common_unigrams_len = len(set(unigrams_que1).intersection(set(unigrams_que2)))
        common_unigrams_lens.append(common_unigrams_len)
        common_unigrams_ratios.append(float(common_unigrams_len) / max(len(set(unigrams_que1).union(set(unigrams_que2))),1))
        
        # get bigram features #
        bigrams_que1 = [i for i in ngrams(unigrams_que1, 2)]
        bigrams_que2 = [i for i in ngrams(unigrams_que2, 2)]
        common_bigrams_len = len(set(bigrams_que1).intersection(set(bigrams_que2)))
        common_bigrams_lens.append(common_bigrams_len)
        common_bigrams_ratios.append(float(common_bigrams_len) / max(len(set(bigrams_que1).union(set(bigrams_que2))),1))


        # get trigram features #
        trigrams_que1 = [i for i in ngrams(unigrams_que1, 3)]
        trigrams_que2 = [i for i in ngrams(unigrams_que2, 3)]
        common_trigrams_len = len(set(trigrams_que1).intersection(set(trigrams_que2)))
        common_trigrams_lens.append(common_trigrams_len)
        common_trigrams_ratios.append(float(common_trigrams_len) / max(len(set(trigrams_que1).union(set(trigrams_que2))),1))

        dif_len.append(abs(len(source_sentence) - len(target_sentence)))
        
    
    
#     # Cosine Similarity on TF IDF vectors
    texts = sentences

#     tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, #max_features=n_features,
#                                    stop_words='english')
#     tfidf_sentences= tfidf_vectorizer.fit_transform(data_set.flatten())
    
#     for i in range(data_set.shape[0]):
#         tfidf_cosin_sim.append(cosine_similarity(tfidf_sentences[2*i,:], tfidf_sentences[2*i+1,:])[0,0])
  
    
    
    #LSI similarities

    #dictionary = corpora.Dictionary(texts)
    #corpus = [dictionary.doc2bow(text) for text in texts]
    #for i in range(data_set.shape[0]):
    #    similarities.append(index[lsi[corpus[2*i]]][2*i+1])
    

    # examples as rows, features as columns
    features = np.array([#tfidf_cosin_sim,
                         #similarities,
                         common_unigrams_lens,
                         common_unigrams_ratios,
                         common_bigrams_lens,
                         common_bigrams_ratios,
                         common_trigrams_lens,
                         common_trigrams_ratios,
                         dif_len]).T
    
    
    # scale
    #normalized_features = preprocessing.scale(features) 
    
    
    
    
    return features

In [ ]:
basic_features = select_basic_features_test(clean_data_test)
new_features_test = test_comb[['q1_hash','q2_hash','q1_freq','q2_freq']].values

In [ ]:
print(basic_features.shape,new_features_test.shape)
X_submission = np.column_stack([basic_features,new_features_test])
X_submission = preprocessing.scale(X_submission)
y_submission = clf.predict_proba(X_submission)[:,1]
Testing_Set['is_duplicate'] = y_submission

In [ ]:
submission = Testing_Set[['test_id','is_duplicate']]

In [ ]:
print(submission)

In [ ]:
submission = submission.to_csv('/Users/pascalsitbon/work/Kaggle/pred.csv',index=False)